In [1]:
url = "https://www.youtube.com/watch?v=XVJaqRSrEZA"

In [2]:
import re

re.findall(r"(?:v=)([\w-]{11})", url)

['XVJaqRSrEZA']

In [3]:
import re

def extract_youtube_video_id(url):
    # Regular expression to capture the video ID
    pattern = r'(?:v=|\/)([0-9A-Za-z_-]{11}).*'
    match = re.search(pattern, url)
    if match:
        return match.group(1)
    return None


In [4]:
pattern = r'(?:v=|\/)([0-9A-Za-z_-]{11}).*'
re.search(pattern, "https://www.youtube.com/watch?v=29XE10U6o-oc&t=6s")


<re.Match object; span=(30, 49), match='v=29XE10U6o-oc&t=6s'>

In [5]:
re.search(pattern, "https://www.youtube.com/watch?v=uyiG6uw-6pA&t=230s").group(1)

'uyiG6uw-6pA'

In [6]:
extract_youtube_video_id("https://www.youtube.com/watch?v=29XE10U6ooc&t=6s")

'29XE10U6ooc'

In [7]:
from youtube_transcript_api import YouTubeTranscriptApi

# Replace with the YouTube video ID
video_id = "ZPUtA3W-7_I"
# video_id = "giqjHxdN-Ok"
# video_id = "95wUTeGHG1k"

In [8]:
# ytt_api = YouTubeTranscriptApi()
# transcript_list = ytt_api.fetch(video_id)

In [9]:
ytt_api = YouTubeTranscriptApi()
transcript_list = ytt_api.get_transcript(video_id, languages=['en','hi'])

In [10]:
transcript_list[:5]

[{'text': '- My strength lies not in my name,',
  'start': 0.0,
  'duration': 3.033},
 {'text': 'but in the backing of 1.4 billion Indians,',
  'start': 4.266,
  'duration': 2.867},
 {'text': 'and thousands of years of timeless culture\nand heritage.',
  'start': 7.6,
  'duration': 3.0},
 {'text': 'So, wherever I go, I carry with me the\nessence of',
  'start': 11.8,
  'duration': 4.366},
 {'text': 'thousands of years of Vedic tradition, the\ntimeless teachings of Swami Vivekananda,',
  'start': 16.266,
  'duration': 5.767}]

In [11]:
type(transcript_list)

list

In [12]:
temp_ = ""
for i in transcript_list:
    temp_ += i['text']
len(temp_)

135379

In [13]:
[True for i in temp_.split(". ") if len(i) > 500]

[True,
 True,
 True,
 True,
 True,
 True,
 True,
 True,
 True,
 True,
 True,
 True,
 True,
 True,
 True,
 True,
 True,
 True,
 True,
 True,
 True,
 True,
 True,
 True,
 True,
 True,
 True,
 True,
 True,
 True,
 True,
 True,
 True,
 True,
 True,
 True,
 True]

In [14]:
68.76+3.719

72.479

In [15]:

chunk_size = 500
chunks = []
timestamps = []

chunk_stack = ""
timestamp_stack = ""

for i in transcript_list:
    the_text = i['text'].strip()
    the_timestamp = i['start']
        
    # if the text is longer than the chunk size
    if len(the_text) >= chunk_size:

        # if previously any chunks exist, append them to the list
        if chunk_stack:
            chunks.append(chunk_stack)
            timestamps.append(timestamp_stack)

            # reset the stack
            chunk_stack = ""
            timestamp_stack = 0

        # append the current chunk
        chunks.append(the_text)
        timestamps.append(the_timestamp)


    # if the text is shorter than the chunk size
    else:
        
        # if chunk and text combined is longer than the chunk size
        if len(chunk_stack := chunk_stack + " " + the_text) > chunk_size:
            
            # split the chunk stack into sentences
            splits = chunk_stack.split(". ")
            temp_chunk_stack = ""
            
            # while the chunk stack is longer than the chunk size
            while(len(chunk_stack := ". ".join(splits)) > chunk_size):
                
                # pop the last sentence from the splits and add it to the temp chunk stack
                temp_chunk_stack = splits.pop() + ". " + temp_chunk_stack
            
            # append the chunk stack and timestamp when the chunk stack is shorter than the chunk size
            chunks.append(chunk_stack.strip())
            timestamps.append(timestamp_stack)

            # reset the stack to temp chunk stack
            chunk_stack = temp_chunk_stack

        # always update the timestamp    
        timestamp_stack = the_timestamp


In [16]:
len(chunks)

317

In [17]:
len(timestamps)

317

ChromaDB

In [18]:
import chromadb
chroma_client = chromadb.PersistentClient("./my_chroma_db")

In [19]:
chroma_client.list_collections()

[Collection(name=youtube_transcripts)]

In [20]:
# generating embeddings

from google import genai
from dotenv import load_dotenv
import os
load_dotenv()
gemini_api_key = os.getenv("GOOGLE_API_KEY")

client = genai.Client(api_key=gemini_api_key)

class GeminiEmbedding:
    def __call__(self, input):
        result = client.models.embed_content(
            model="models/text-embedding-004",
            contents=input)

        return [e.values for e in result.embeddings]
    def name(self):
        return "gemini_embedding"

In [21]:
collection = chroma_client.get_or_create_collection("youtube_transcripts",embedding_function=GeminiEmbedding())

In [23]:
import numpy as np
import uuid

chunks_len = len(chunks)
for i in range(0,chunks_len,100):

    collection.add(
        documents=chunks[i:i+100],
        ids=[str(uuid.uuid4()) for _ in range(100 if chunks_len >= 100 else chunks_len%100)],
        metadatas=[{
            "time_stamp": t,
            "youtube_id": video_id,
        } for t in timestamps[i:i+100] ]
    )
    chunks_len -= 100

In [24]:
len(collection.get()['ids'])

317

In [34]:
iiii = collection.query( query_texts='pollution in India',n_results=10,include=['documents','metadatas'])

In [40]:
iiii['documents'][0]

["He joined the struggle for India’s\nindependence..  And to this day, he continues to deeply\ninfluence the life of every Indian in some\nway. Mahatma Gandhi tried to live by his\nprinciples, and practiced what he\npreached. For example, he strongly advocated for\ncleanliness and practiced it himself. And he made it a point to discuss\ncleanliness wherever he went. Another key factor to consider is India's\nfight for independence",
 "Whenever we speak of peace, the world\nlistens to us..  Because India is the land of Gautama\nBuddha and Mahatma Gandhi. And Indians aren't hardwired to espouse\nstrife and conflict. We espouse harmony instead. We seek neither to wage war against\nnature, nor to foster strife among\nnations. We stand for peace. And wherever we can act as peacemakers, we\nhave gladly embraced that responsibility. My early life was\nspent in extreme poverty. But we never really felt the burden of\npoverty",
 'Both nations have ancient cultures and\ncivilizations..  Even in 

In [42]:
[i['time_stamp'] for i in iiii['metadatas'][0]]

[3975.366,
 85.333,
 6126.533,
 4012.666,
 3762.966,
 9170.0,
 3675.2,
 4659.9,
 3850.066,
 9434.633]

In [63]:
kkkk = collection.query( query_texts='pollution in India', n_results=5, where={"youtube_id":"PUtA3W-7_I"})

In [64]:
kkkk

{'ids': [[]],
 'embeddings': None,
 'documents': [[]],
 'uris': None,
 'included': ['metadatas', 'documents', 'distances'],
 'data': None,
 'metadatas': [[]],
 'distances': [[]]}

In [67]:
if kkkk['documents'][0]:
    print("yes")

In [59]:
collection.get()

{'ids': ['cc3feec8-9ad3-461d-a003-22c727b1567e',
  '2152e66b-9471-4517-8305-58ad98564c5f',
  'ae55d49f-34eb-418f-892b-64e6636c3c98',
  '8c7bbcca-1c6f-4fb9-8d9d-3aac35b68dca',
  'cccb804d-c1d6-4274-bf0d-f80ce849f48b',
  '57d0451f-8885-4ca2-a349-ca2d08f91161',
  'b276902d-c6f5-4797-a1b8-3aa54205105f',
  '7b211930-ae99-41cc-8069-2ce85cc0a849',
  '72669d7e-4e59-4a20-960f-f6c40a050cd1',
  '21eb9f70-8f91-4d12-b6bb-380c6935abf5',
  '6b85288d-2c28-4efe-8071-1e67acab8912',
  '061a366a-a44c-4e14-b0d8-a84befb558d7',
  'd7c00091-f882-489f-86cb-bd4795ee9d12',
  '77880336-d4f9-4f90-a58f-5fc1d4cec002',
  'db848d2c-faf7-40a0-b752-566b691a5038',
  'f1f02913-d2eb-4dc3-9b3f-e57df85d9466',
  '0c6d77a3-ed20-4573-bba9-4cab557c0b5e',
  '48bb7f0c-e69d-4dcc-b017-eb293c43cb46',
  '213fca59-7e6f-4d45-8db2-c80ee32cfeb7',
  '42aeed6d-1fe5-4163-ac3f-947a5ffecad2',
  'a22709fc-3185-4d99-98db-46e5829ee73e',
  'caf5e128-6665-4943-959e-fa6864dcdbf1',
  '7041608e-7e96-4d07-979d-c438da32b77a',
  'd25056ee-f6b2-4ec9-a3c6-

In [33]:
kkkk

[["He joined the struggle for India’s\nindependence..  And to this day, he continues to deeply\ninfluence the life of every Indian in some\nway. Mahatma Gandhi tried to live by his\nprinciples, and practiced what he\npreached. For example, he strongly advocated for\ncleanliness and practiced it himself. And he made it a point to discuss\ncleanliness wherever he went. Another key factor to consider is India's\nfight for independence",
  "Whenever we speak of peace, the world\nlistens to us..  Because India is the land of Gautama\nBuddha and Mahatma Gandhi. And Indians aren't hardwired to espouse\nstrife and conflict. We espouse harmony instead. We seek neither to wage war against\nnature, nor to foster strife among\nnations. We stand for peace. And wherever we can act as peacemakers, we\nhave gladly embraced that responsibility. My early life was\nspent in extreme poverty. But we never really felt the burden of\npoverty",
  'Both nations have ancient cultures and\ncivilizations..  Even 

In [ ]:
iiii

{'ids': [['6f7a2e26-0ce0-4480-a7e3-75c254d319bd',
   '718108fd-2e8a-422d-b58b-2a093416f04b',
   'dab0f610-23d3-4e7d-ad85-ab3119b872f0',
   '5876c90e-6673-48f5-b323-6651653d9c5c',
   '80e0987b-fd3e-45fe-b9d6-a6d3286737ff',
   '22a50694-46a9-4a70-8b9f-332809b1c775',
   'f6581c15-cf3a-46b2-afc2-eeeb06d8638a',
   '1aac7aa1-91f2-4aa3-8ea0-f8e236515302',
   'afaeaf26-2e84-429e-ba26-07b1139c9fde',
   '22b3dc01-51b3-47c4-a172-a7ebebfa0796']],
 'embeddings': None,
 'documents': [["Um next topic which is somewhat related is that the air quality in India we we recently went to India I think it was in December and it's the opposite of HPO because the pollution there was so bad that we almost canled the trip last.  minute. I think sometimes like the the levels were hundreds of times higher than the recommended uh PM PMI. Is that what it is? PMI limits. Yeah. Am I? Yeah. PMI. PM2.5. PM2.5. Yeah. Yeah. PM PM2.5. Yeah",
   "And it carries these toxins, these nitrogen, nitrogen oxides, sulfur dioxides,

In [ ]:
client.models.embed_content(
        model="models/text-embedding-004",
        contents='who are you').embeddings[0].values